# SARIMAX Forecasting: Train on `train.csv`, Forecast on `test.csv`

**Workflow**:
1. Import libraries.
2. Configure paths & parameters.
3. Read `train.csv` (with `sales`) and `test.csv` (future dates, no `sales`).
4. Loop over each product family:
   - Aggregate daily `sales` & `onpromotion` from `train.csv`.
   - (Optional) Hold out last `N_VALID` days of `train.csv` for sanity‐check metrics.
   - Log1p‐transform `sales` and scale both the transformed `sales` and `onpromotion`.
   - Fit SARIMAX on the scaled target/exogenous series (orders selected via `auto_arima`).
   - Save model to `Model3/`.
   - Forecast exactly on the dates present in `test.csv` (using its scaled `onpromotion`).
5. (Optional) Compute RMSE/MAE/R²/MAPE on held‐out portion of `train.csv` and save to `Performance/performance_all_families.csv`.
6. Save all forecasts (for `test.csv`) into `forecasts_all_families.csv`.

## Import all libraries and suppress warnings

In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pmdarima import auto_arima
from IPython.display import display

warnings.filterwarnings("ignore")

## Set all paths, parameters, and output directories

In [ ]:
# 1) File paths (adjust if needed)
TRAIN_CSV = "../data/DM/train_cleaned.csv"
TEST_CSV  = "../data/DM/test.csv"

# 2) Output directories / filenames
MODEL_DIR   = os.joinpath("Models", "Model3")
PERF_DIR    = "Performance"
FORE_DIR    = "Forecasts"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(PERF_DIR, exist_ok=True)
os.makedirs(FORE_DIR, exist_ok=True)

PERF_CSV     = os.path.join(PERF_DIR, "performance_all_families.csv")
FORECAST_CSV = os.path.join(FORE_DIR, "forecasts_all_families.csv")

# 3) Hold-out size (set to 0 to skip hold-out)
N_VALID = 14

# 4) Time-series frequency
FREQ = "D"  # daily

# 5) auto_arima arguments
AUTO_ARIMA_ARGS = {
    "seasonal": True,
    "m": 7,
    "start_p": 0, "start_q": 0, "max_p": 5, "max_q": 5,
    "start_P": 0, "start_Q": 0, "max_P": 2, "max_Q": 2,
    "d": None, "D": None,
    "trace": False,
    "error_action": "ignore",
    "suppress_warnings": True,
    "stepwise": True,
    "information_criterion": "aic"
}

## CELL 3: Load train and test CSVs, list families

In [3]:
print("Loading train.csv and test.csv...")
df_train_all = pd.read_csv(TRAIN_CSV, parse_dates=["date"])
df_test_all  = pd.read_csv(TEST_CSV,  parse_dates=["date"])

print(f"→ train.csv: {df_train_all.shape[0]} rows")
print(f"→ test.csv:  {df_test_all.shape[0]} rows\n")

unique_families = df_train_all["family"].unique()
print(f"Found {len(unique_families)} unique families.\n")
display(unique_families)

Loading train.csv and test.csv...
→ train.csv: 2061758 rows
→ test.csv:  28512 rows

Found 33 unique families.



array(['BEAUTY', 'BEVERAGES', 'BREAD/BAKERY', 'CLEANING', 'DAIRY', 'DELI',
       'EGGS', 'FROZEN FOODS', 'GROCERY I', 'GROCERY II',
       'LAWN AND GARDEN', 'LINGERIE', 'LIQUOR,WINE,BEER', 'MEATS',
       'PERSONAL CARE', 'POULTRY', 'PREPARED FOODS', 'AUTOMOTIVE',
       'HARDWARE', 'SEAFOOD', 'HOME APPLIANCES', 'PRODUCE', 'CELEBRATION',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME CARE',
       'MAGAZINES', 'PET SUPPLIES', 'PLAYERS AND ELECTRONICS',
       'LADIESWEAR', 'SCHOOL AND OFFICE SUPPLIES', 'BABY CARE', 'BOOKS'],
      dtype=object)

## CELL 4: Define helper for Augmented Dickey-Fuller stationarity test

In [4]:
def test_stationarity(ts, name="series"):
    result = adfuller(ts)
    print("  ─" * 30)
    print(f"  ADF Statistic ({name}): {result[0]:.5f}")
    print(f"  p-value:            {result[1]:.5f}")
    for key, val in result[4].items():
        print(f"  Critical Value ({key}): {val:.5f}")
    if result[1] < 0.05:
        print("  → The series IS stationary (reject H₀).")
    else:
        print("  → The series is NOT stationary (fail to reject H₀).")
    print("  ─" * 30)

## CELL 5: Main loop – Process each family

In [5]:
performance_records = []
forecast_records    = []

for family in unique_families:
    print("\n" + "=" * 60)
    print(f"Processing family = '{family}'")
    
    # 4a) Filter train for this family and aggregate by date
    df_f = df_train_all[df_train_all["family"] == family].copy()
    if df_f.empty:
        print(f"  No rows for '{family}' in train.csv. Skipping.\n")
        continue

    df_f_agg = (
        df_f.groupby("date")[["sales", "onpromotion"]]
           .sum()
           .rename(columns={"onpromotion": "onpromo"})
           .sort_index()
    )

    # Convert to daily PeriodIndex and fill missing dates with zeros
    df_f_agg.index = pd.DatetimeIndex(df_f_agg.index).to_period(FREQ)
    df_f_agg = df_f_agg.asfreq(FREQ)
    df_f_agg["sales"]   = df_f_agg["sales"].fillna(0).astype(float)
    df_f_agg["onpromo"] = df_f_agg["onpromo"].fillna(0).astype(float)

    # 4b) Remove zero-sales rows
    orig_len = len(df_f_agg)
    df_f_agg = df_f_agg[df_f_agg["sales"] != 0].copy()
    print(f"  Removed {orig_len - len(df_f_agg)} zero-sales rows → {len(df_f_agg)} remaining")

    # Ensure enough data points for hold-out
    if len(df_f_agg) < (N_VALID + 1):
        print(f"  Not enough data points ({len(df_f_agg)}) for hold-out of {N_VALID}. Skipping.\n")
        continue

    # 4c) Hold out last N_VALID days
    if N_VALID > 0:
        df_valid_holdout = df_f_agg.iloc[-N_VALID:]
        df_train_series  = df_f_agg.iloc[:-N_VALID]
        print(f"  Held out last {N_VALID} days for validation.")
    else:
        df_valid_holdout = None
        df_train_series  = df_f_agg

    # 4d) Stationarity test on raw sales (before log)
    print("  Running ADF test on training 'sales' series:")
    test_stationarity(df_train_series["sales"], name="sales")

    # 4e) Log-transform the sales column
    df_train_series["y_log"] = np.log1p(df_train_series["sales"])

    # 4f) Scale the log-transformed sales
    y_log_raw    = df_train_series["y_log"].values.reshape(-1, 1)
    scaler_y     = StandardScaler()
    y_log_scaled = scaler_y.fit_transform(y_log_raw).ravel()
    df_train_series["y_scaled"] = y_log_scaled

    # 4g) Prepare and scale exogenous (onpromo)
    exog_train_raw = df_train_series["onpromo"].values.reshape(-1, 1)
    if df_valid_holdout is not None:
        exog_valid_raw = df_valid_holdout["onpromo"].values.reshape(-1, 1)

    scaler_x = StandardScaler()
    exog_train_scaled = scaler_x.fit_transform(exog_train_raw).ravel()
    exog_train = pd.Series(exog_train_scaled, index=df_train_series.index)

    if df_valid_holdout is not None:
        exog_valid_scaled = scaler_x.transform(exog_valid_raw).ravel()
        exog_valid = pd.Series(exog_valid_scaled, index=df_valid_holdout.index)
    else:
        exog_valid = None

    # 4h) Auto-arima on the scaled target with scaled exogenous
    print("  Running auto_arima for hyperparameter tuning...")
    try:
        mi = auto_arima(
            df_train_series["y_scaled"],
            exogenous=exog_train.values.reshape(-1, 1),
            **AUTO_ARIMA_ARGS
        )
        order_opt          = mi.order
        seasonal_order_opt = mi.seasonal_order
        print(f"   → Selected order = {order_opt}, seasonal_order = {seasonal_order_opt}")
    except Exception as e:
        print(f"   ❗ auto_arima failed for '{family}': {e}\n")
        continue

    # 4i) Fit SARIMAX on the scaled target and scaled exogenous
    print("  Fitting SARIMAX on scaled series with scaled exogenous...")
    try:
        model = SARIMAX(
            df_train_series["y_scaled"],
            exog=exog_train,
            order=order_opt,
            seasonal_order=seasonal_order_opt,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        sarimax_fit = model.fit(disp=False)
    except Exception as e:
        print(f"   ❗ SARIMAX fit error for '{family}': {e}\n")
        continue

    # 4j) Save the fitted model to Model3/
    safe_name  = family.replace(" ", "_").replace("/", "_").replace("\\", "_")
    model_file = f"sarimax_family_{safe_name}.pkl"
    model_path = os.path.join(MODEL_DIR, model_file)
    try:
        sarimax_fit.save(model_path)
        print(f"   → Saved model to {model_path}")
    except Exception as e:
        print(f"   ❗ Could not save model for '{family}': {e}")

    # 5) Compute hold-out metrics (if applicable)
    if df_valid_holdout is not None:
        print("  Computing hold-out metrics:")
        # Forecast on scaled hold-out exogenous
        # NOTE: convert Series to np.array before reshape
        pred_scaled_valid = sarimax_fit.get_forecast(
            steps=N_VALID,
            exog=exog_valid.values.reshape(-1, 1)
        ).predicted_mean

        # Invert scaling and log-transform to get predictions on original sales scale
        pred_log_valid = scaler_y.inverse_transform(pred_scaled_valid.values.reshape(-1, 1)).ravel()
        pred_valid     = np.expm1(pred_log_valid)
        actual_valid   = df_valid_holdout["sales"].values

        rmse_val = np.sqrt(mean_squared_error(actual_valid, pred_valid))
        mae_val  = mean_absolute_error(actual_valid, pred_valid)
        r2_val   = r2_score(actual_valid, pred_valid)
        mape_val = np.mean(np.abs((actual_valid - pred_valid) / actual_valid)) * 100

        print(f"   → RMSE = {rmse_val:.2f}, MAE = {mae_val:.2f}, R² = {r2_val:.3f}, MAPE = {mape_val:.2f}%")
        performance_records.append({
            "family": family,
            "RMSE": rmse_val,
            "MAE": mae_val,
            "R2": r2_val,
            "MAPE": mape_val
        })

    # 6) Forecast on test set for this family
    df_test_sub = df_test_all[df_test_all["family"] == family].copy()
    if df_test_sub.empty:
        print(f"  No rows for '{family}' in test.csv. Skipping forecast.\n")
        continue

    df_test_sub["date"] = pd.to_datetime(df_test_sub["date"])
    agg_exog_test = (
        df_test_sub.groupby("date")[["onpromotion"]]
                   .sum()
                   .rename(columns={"onpromotion": "onpromo"})
    )
    agg_exog_test.index = pd.DatetimeIndex(agg_exog_test.index).to_period(FREQ)
    agg_exog_test = agg_exog_test.asfreq(FREQ, fill_value=0)

    exog_test_raw   = agg_exog_test["onpromo"].values.reshape(-1, 1)
    exog_test_scaled = scaler_x.transform(exog_test_raw).ravel()

    h = len(agg_exog_test)
    print(f"  Forecasting {h} future points for '{family}' on test set...")
    try:
        pred_scaled_test = sarimax_fit.get_forecast(
            steps=h,
            exog=exog_test_scaled.reshape(-1, 1)
        ).predicted_mean

        pred_log_test   = scaler_y.inverse_transform(pred_scaled_test.values.reshape(-1, 1)).ravel()
        pred_sales_test = np.expm1(pred_log_test)
    except Exception as e:
        print(f"   ❗ Forecast error for '{family}': {e}\n")
        continue

    # Build date-level forecast DataFrame
    df_forecast_dates = pd.DataFrame({
        "date": agg_exog_test.index.to_timestamp(),
        "predicted_sales": pred_sales_test
    })

    # Broadcast forecasts to each store in the test subset
    df_test_sub = df_test_sub.merge(
        df_forecast_dates,
        left_on="date",
        right_on="date",
        how="left"
    )

    df_forecast_f = pd.DataFrame({
        "id": df_test_sub["id"].values,
        "date": df_test_sub["date"].values,
        "store_nbr": df_test_sub["store_nbr"].values,
        "family": family,
        "predicted_sales": df_test_sub["predicted_sales"].values
    })

    forecast_records.append(df_forecast_f)
    print(f"   → Done forecasting for '{family}'.")

# ────────────────────────────────────────────────────────────────────────────────
print("\n" + "=" * 60)
print("All families processed.\n")

# 5) Save performance metrics
if performance_records:
    perf_df = pd.DataFrame(performance_records)
    perf_df.to_csv(PERF_CSV, index=False)
    print(f"Hold-out performance metrics saved to {PERF_CSV}")
    display(perf_df)
else:
    print("No hold-out metrics were generated (N_VALID might be 0).")

# 6) Save all forecasts
if forecast_records:
    all_forecasts_df = pd.concat(forecast_records, axis=0, ignore_index=True)
    all_forecasts_df = all_forecasts_df.sort_values(["family", "date", "store_nbr"]).reset_index(drop=True)
    all_forecasts_df.to_csv(FORECAST_CSV, index=False)
    print(f"All forecasts saved to {FORECAST_CSV}")
    display(all_forecasts_df.head(20))
else:
    print("No forecasts generated (check test.csv).")


Processing family = 'BEAUTY'
  Removed 0 zero-sales rows → 1684 remaining
  Held out last 14 days for validation.
  Running ADF test on training 'sales' series:
  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─
  ADF Statistic (sales): -2.04183
  p-value:            0.26854
  Critical Value (1%): -3.43433
  Critical Value (5%): -2.86330
  Critical Value (10%): -2.56771
  → The series is NOT stationary (fail to reject H₀).
  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─  ─
  Running auto_arima for hyperparameter tuning...
   → Selected order = (3, 1, 1), seasonal_order = (2, 0, 1, 7)
  Fitting SARIMAX on scaled series with scaled exogenous...
   → Saved model to Model3\sarimax_family_BEAUTY.pkl
  Computing hold-out metrics:
   → RMSE = 45.70, MAE = 38.72, R² = 0.658, MAPE = 10.22%
  Forecasting 16 future points for 'BEAUTY' on test set...
   → Done forecasting for 'BEAUTY'.

Processing family = 'BEVERAGES'

,family,RMSE,MAE,R2,MAPE
0,BEAUTY,45.701553,38.717910,0.657927,10.219163
1,BEVERAGES,21451.056731,18345.709094,0.371192,9.752256
2,BREAD/BAKERY,3334.999966,2607.949527,0.151131,8.967921
3,CLEANING,37396.294632,34481.576976,-9.429157,58.803948
4,DAIRY,4771.198453,3597.523202,0.445720,8.092465
5,DELI,1814.385591,1328.488568,0.485269,8.509359
6,EGGS,1686.402971,1443.740985,0.412380,13.708516
7,FROZEN FOODS,747.177253,555.304148,0.801912,7.752965
8,GROCERY I,26215.152853,20713.221745,0.334263,8.403149
9,GROCERY II,496.905122,443.960195,-5.065413,30.032185


All forecasts saved to forecasts_all_families.csv


,id,date,store_nbr,family,predicted_sales
0,3000888,2017-08-16,1,AUTOMOTIVE,319.715565
1,3001251,2017-08-16,2,AUTOMOTIVE,319.715565
2,3001614,2017-08-16,3,AUTOMOTIVE,319.715565
3,3001977,2017-08-16,4,AUTOMOTIVE,319.715565
4,3002340,2017-08-16,5,AUTOMOTIVE,319.715565
5,3002538,2017-08-16,6,AUTOMOTIVE,319.715565
6,3002571,2017-08-16,7,AUTOMOTIVE,319.715565
7,3002604,2017-08-16,8,AUTOMOTIVE,319.715565
8,3002637,2017-08-16,9,AUTOMOTIVE,319.715565
9,3000921,2017-08-16,10,AUTOMOTIVE,319.715565
